# Libraries

In [ ]:
#Importing all the librairies we'll need
import pandas as pd
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeRegressor
from sklearn import linear_model
from sklearn import preprocessing
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
from sklearn.linear_model import LinearRegression,Ridge
from sklearn.model_selection import train_test_split, cross_val_score,KFold
from xgboost import XGBRegressor
from sklearn.ensemble import StackingRegressor,VotingRegressor,BaggingRegressor,ExtraTreesRegressor,RandomForestRegressor,GradientBoostingRegressor
from sklearn.cluster import KMeans



# Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train=pd.read_csv("/content/drive/MyDrive/ZINDI/Cryptocurrency Closing Price Prediction/Train.csv")

In [ ]:
test=pd.read_csv("/content/drive/MyDrive/ZINDI/Cryptocurrency Closing Price Prediction/Test.csv")

In [ ]:
settozero=test.loc[test['open'].isnull()].id

In [ ]:
train=train.loc[train['close'].isnull()==False]

# Modeling

In [ ]:
kfolds=KFold(n_splits=5, shuffle=True, random_state=1994)

In [ ]:
X=train[['high','low','open','market_cap','price_score','volatility','galaxy_score','percent_change_24h_rank','percent_change_24h','market_cap_global','market_cap_rank']]
y = train.close.astype(float)

feat=X.columns
kmeans = KMeans(n_clusters=5, random_state=50)
X["cluster"]=kmeans.fit_predict(X.fillna(0))
test["cluster"]=kmeans.predict(test[feat].fillna(0))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
y_pred_tot=[]
tot_auc=[]
tot=[]
rmse_score=[]
oof = np.zeros(len(X))
models = [('ridge',make_pipeline(preprocessing.RobustScaler(),BaggingRegressor(Ridge(alpha=0.001),n_estimators=50,random_state=50))),
          ('linearregression',make_pipeline(preprocessing.RobustScaler(),BaggingRegressor(linear_model.LinearRegression(),random_state=50,n_estimators=10))),
          ('bayesianopt',make_pipeline(preprocessing.RobustScaler(),BaggingRegressor(linear_model.BayesianRidge(n_iter=1000),random_state=50,n_estimators=10))),
          ('et',make_pipeline(preprocessing.RobustScaler(),ExtraTreesRegressor(random_state=50,n_estimators=220,max_depth=15))),
          ('gb',make_pipeline(preprocessing.RobustScaler(),GradientBoostingRegressor(random_state=50,learning_rate=0.01,loss='ls',n_estimators=1700,max_depth=5))),
          ('rf',make_pipeline(preprocessing.RobustScaler(),RandomForestRegressor(random_state=50,n_estimators=500,max_depth=15))),
          ('xgb',make_pipeline(preprocessing.RobustScaler(),XGBRegressor(objective ='reg:tweedie',n_estimators=2500,learning_rate=0.01,max_depth=15,random_state=50,eval_metric='rmse',reg_lambda=0.5,reg_alpha=0.01,subsample=0.9)))

          ]
X=X.fillna(-1)
for ind,(ind_train,ind_val) in (enumerate (kfolds.split(X,y))): 
    X_train,X_val = X.iloc[ind_train],X.iloc[ind_val] 
    y_train,y_val = y.iloc[ind_train],y.iloc[ind_val]
  
    model =StackingRegressor(estimators=models,final_estimator=BaggingRegressor(linear_model.LinearRegression(),random_state=50,n_estimators=15))
    model.fit(X_train,y_train)
    val_pred = model.predict(X_val)
    oof[ind_val] = val_pred.clip(0)
    score_val=np.sqrt(mean_squared_error(y_val, val_pred))
    print(score_val)
    test_pred = model.predict(test[X.columns].fillna(-1))                                         
    y_pred_tot.append(test_pred)
    rmse_score.append(score_val)
print('*'*25)
print(np.mean(rmse_score,0))

49.35149064399218
52.227968991478384
51.67935641026512
51.34256174309441
67.64652711233626
*************************
54.44958098023326


In [ ]:
oofD = pd.DataFrame(oof,columns=['OOF'])
oofD['id'] = train['id'].values
oofD['close'] = y.values

newcols = []
cols = ['high','low','open']
for col in cols :
  oofD[col] = X[col].values
  oofD[col+'_diff'] = oofD['OOF'] - oofD[col]
  newcols.append(col+'_diff')

for i in oofD.index:
  oofD['OOF'][i]=np.clip(oofD['OOF'][i],oofD['low'][i], oofD['high'][i])
oofD['diff'] = oofD['close'] - oofD['OOF']
oofD = oofD[['id','OOF','close','diff']+cols+newcols]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [ ]:
mean_squared_error(oofD['close'],oofD['OOF'],squared=False)

53.83129861829804

In [ ]:
mean_squared_error(oofD['close'],(oofD['OOF']*1.09-oofD['high']*0.04-oofD['low']*0.04-oofD['open']*0.01),squared=False)

53.44813267092467

# Submission

In [ ]:
len(y_pred_tot)

5

In [ ]:
predictions=np.mean(y_pred_tot,0)
sub_file = test[['id','low','open','high']].fillna(0)
sub_file['close']= predictions


In [ ]:
predictions.shape

(6222,)

In [ ]:
sub_file

,id,low,open,high,close
0,ID_323Sn2,0.000000,0.000000,0.000000,3707.572266
1,ID_325SNW,11335.062188,11335.062188,11351.690956,11351.819759
2,ID_325uzE,6294.714484,6322.560756,6328.362354,6308.574031
3,ID_328qCx,0.000000,0.000000,0.000000,3707.572266
4,ID_3293uJ,0.000000,0.000000,0.000000,3707.572266
...,...,...,...,...,...
6217,ID_zufSPk,8259.796103,8266.942912,8266.942912,8265.512515
6218,ID_zuz9yf,10865.827625,10869.612767,10873.493490,10872.361277
6219,ID_zvrMSX,0.000000,0.000000,0.000000,3707.572266
6220,ID_zy9Cfv,0.000000,0.000000,0.000000,3707.572266


In [ ]:
for i in sub_file.index:
  sub_file['close'][i]=np.clip(sub_file['close'][i],sub_file['low'][i], sub_file['high'][i])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
for i in settozero:
  sub_file.loc[sub_file['id']==i,'close']=0

In [ ]:
mean_squared_error(oofD['close'],(oofD['OOF']*1.09-oofD['high']*0.04-oofD['low']*0.04-oofD['open']*0.01),squared=False)

53.44813267092467

In [ ]:
sub_file['close'] = sub_file['close']*1.09-sub_file['high']*0.04-sub_file['low']*0.04-sub_file['open']*0.01

In [ ]:
sub_file[['id','close']].to_csv('submitOOF53.44.csv', index = False)